# Matemática da Arquitetura Transformer na Análise e Forecast de Séries Temporais

https://arxiv.org/abs/1706.03762

HuggingFace como fonte de dados e do modelo pré-treinado.

https://huggingface.co/

## Instalar e carregar pacotes

In [ ]:
!pip install -q transformers==4.37.2

In [ ]:
!pip install -q datasets==2.16.1

In [ ]:
!pip install -q evaluate==0.4.1

In [ ]:
!pip install -q accelerate==0.26.1

In [ ]:
!pip install -q -U gluonts==0.16.2

https://ts.gluon.ai

In [ ]:
!pip install -q ujson==5.4.0

In [ ]:
!pip install -q urllib3==1.26.16

In [ ]:
%env TF_CPP_MIN_LOG_LEVEL=3

In [ ]:
# Ajusta problema de performance do TensorFlow em CPUs Intel
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [ ]:
# Ajusta problema de performance do PyTorch
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

In [ ]:
# Imports
import ujson
import urllib3
import evaluate
import torch
import transformers
import accelerate
import gluonts
import pandas as pd
import numpy as np
from datasets import load_dataset
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from functools import lru_cache
from functools import partial
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerForPrediction
from transformers import PretrainedConfig
from typing import Optional
from accelerate import Accelerator
from torch.optim import AdamW
from evaluate import load
from typing import Iterable
from gluonts.itertools import Cached, Cyclic
from gluonts.dataset.loader import as_stacked_batches
from gluonts.time_feature import get_seasonality
from gluonts.time_feature import get_lags_for_frequency
from gluonts.time_feature import time_features_from_frequency_str
from gluonts.transform.sampler import InstanceSampler
from gluonts.time_feature import (time_features_from_frequency_str, TimeFeature, get_lags_for_frequency)
from gluonts.dataset.field_names import FieldName
from gluonts.transform import (
    AddAgeFeature,
    AddObservedValuesIndicator,
    AddTimeFeatures,
    AsNumpyArray,
    Chain,
    ExpectedNumInstanceSampler,
    InstanceSplitter,
    RemoveFields,
    SelectFields,
    SetField,
    TestSplitSampler,
    Transformation,
    ValidationSplitSampler,
    VstackFeatures,
    RenameFields,
)
import warnings
warnings.filterwarnings('ignore')

## Dados
https://huggingface.co/datasets/Monash-University/monash_tsf

In [ ]:
dataset = load_dataset("monash_tsf", "tourism_monthly")
dataset

In [ ]:
dataset['train']

In [ ]:
exemplo_treino = dataset['train'][0]
type(exemplo_treino)

In [ ]:
exemplo_treino.keys()

In [ ]:
print(exemplo_treino['start'])
print(exemplo_treino['target'])

In [ ]:
exemplo_valid = dataset['validation'][0]
exemplo_valid.keys()

In [ ]:
print(exemplo_valid['start'])
print(exemplo_valid['target'])

In [ ]:
# Vamos extrair um elemento de teste
exemplo_teste = dataset['test'][0]

In [ ]:
print(exemplo_teste['start'])
print(exemplo_teste['target'])

In [ ]:
len(exemplo_treino['target'])

In [ ]:
len(exemplo_valid['target'])

In [ ]:
len(exemplo_teste['target'])

## Visualização da série temporal

In [ ]:
# Frequência da série temporal (1 mês)
freq = "1M"

# Janela de previsão (24 meses)
prediction_length = 24

In [ ]:
# Verifica se o comprimento dos dados de validação permite a janela de previsão
assert len(exemplo_treino["target"]) + prediction_length == len(exemplo_valid["target"])

In [ ]:
# Plot
fig, ax = plt.subplots()
ax.plot(exemplo_treino['target'], label='Treino')
ax.plot(exemplo_valid["target"], color = "red", alpha = 0.5)
plt.show()

## Converter o formato de dados

In [ ]:
dataset_treino = dataset['train']
dataset_teste = dataset['test']

In [ ]:
# Função para converter datas para períodos de datas
def convert_to_period(date, freq: str):
    return pd.Period(date, freq)

In [ ]:
# Função para definir o início do batch de dados
def define_start(batch, freq):
    batch["start"] = [convert_to_period(date, freq) for date in batch["start"]]
    return batch

In [ ]:
# Ajusta os datasets das séries temporais no formato apropriado
dataset_treino.set_transform(partial(define_start, freq = freq))
dataset_teste.set_transform(partial(define_start, freq = freq))

## TimeSeries Transformer Config

https://huggingface.co/docs/transformers/en/model_doc/time_series_transformer

A configuração define vários hiperparâmetros para o modelo. Descrição de cada hiperparâmetro:

`prediction_length`: O número de etapas à frente que o modelo deve prever. Por exemplo, se você estiver trabalhando com dados diários e quiser prever uma semana à frente, o prediction_length seria 7.

`context_length`: O número de pontos de dados anteriores que o modelo usará para fazer suas previsões. Neste caso, é definido como duas vezes o prediction_length.

`lags_sequenc`: Esta é a sequência de atrasos de tempo (ou "lags") que o modelo usará. Lags são valores passados em uma série temporal. Por exemplo, em um modelo de regressão, você pode usar dados de um dia, uma semana e um mês atrás como entradas.

`num_time_features`: O número de características temporais que o modelo usará. Características de tempo podem incluir itens como a hora do dia, dia da semana, mês do ano, etc.

`num_static_categorical_features`: O número de características categóricas estáticas. Essas são características que não mudam com o tempo, como o ID de uma loja ou produto em previsões de vendas.

`cardinality`: O número de valores possíveis para cada característica categórica. Aqui, há uma característica categórica com 366 valores possíveis.

`embedding_dimension`: A dimensionalidade do espaço de incorporação para as características categóricas. Neste caso, cada um dos 366 possíveis valores categóricos será mapeado para um vetor de 2 dimensões.

`encoder_layers` e `decoder_layers`: O número de camadas na codificador e decodificador do transformador, respectivamente.

`d_model`: A dimensionalidade do espaço de entrada e saída para o transformador.


Todos esses parâmetros serão usados para construir e treinar o modelo de transformador de séries temporais. A escolha desses hiperparâmetros pode ter um grande impacto na performance do modelo.


In [ ]:
# TimeSeries Transformer Config
config = TimeSeriesTransformerConfig(

    # Comprimento de previsão
    prediction_length = prediction_length,

    # Comprimento do contexto
    context_length = prediction_length * 2,

    # Lags sequence
    # "Lags" em séries temporais referem-se a pontos de dados anteriores em uma série de tempo.
    # Em outras palavras, um "lag" é um atraso temporal. Por exemplo, em uma série temporal mensal, o "lag"
    # de um mês refere-se aos dados do mês anterior.
    lags_sequence = get_lags_for_frequency(freq),

    # Adicionaremos 2 características de tempo ("mês do ano" e "idade da série"):
    num_time_features = len(time_features_from_frequency_str(freq)) + 1,

    # Temos um único recurso categórico estático, ou seja, o ID da série temporal
    num_static_categorical_features = 1,

    # Temos 366 valores possíveis
    cardinality = [len(dataset_treino)],

    # O modelo receberá uma embedding de tamanho 2 para cada um dos 366 valores possíveis:
    embedding_dimension = [2],

    # Parâmetros da rede neural do Transformer
    encoder_layers = 4,
    decoder_layers = 4,
    d_model = 32,
)

### Matemática do Modelo Transformer

https://arxiv.org/pdf/1706.03762.pdf

Vamos descrever as partes do modelo Transformer com algumas fórmulas. Os termos a seguir serão úteis para entender os cálculos.

- Q: vetor de consulta
- K: vetor de chave
- V: vetor de valor

As fórmulas para o mecanismo de atenção no Transformer são as seguintes:

1- Atenção Escalada por Produto Escalar

A função de atenção é usada para calcular a importância de diferentes partes da entrada. Ela recebe três entradas: Q, K e V. A saída é calculada como:

**Attention(Q, K, V) = softmax((QK^T) / sqrt(d_k))V**

onde d_k é a dimensão dos vetores-chave e o operador ^T indica a transposição de uma matriz. A operação de produto escalar entre Q e K ajuda a determinar a relevância entre cada par de consulta e chave. O resultado é então dividido pela raiz quadrada de d_k para evitar que os valores do produto escalar fiquem muito grandes. Por fim, a função softmax é aplicada para transformar os pesos em probabilidades que somam 1. Esses pesos são então usados para ponderar os vetores de valor.

2- Atenção Multi-cabeça

A atenção multi-cabeça permite que o modelo se concentre em diferentes partes da entrada para cada cabeça de atenção. Suponha que temos h cabeças de atenção. Para cada cabeça, primeiro transformamos Q, K e V com diferentes pesos aprendidos:

- Q_i = QW^Q_i
- K_i = KW^K_i
- V_i = VW^V_i

onde W^Q_i, W^K_i e W^V_i são os pesos aprendidos para a i-ésima cabeça.

Em seguida, aplicamos a atenção escalada por produto escalar para cada conjunto de Q_i, K_i e V_i:

**head_i = Attention(Q_i, K_i, V_i)**

A saída de todas as cabeças é então concatenada e linearmente transformada para produzir a saída final:

**MultiHead(Q, K, V) = Concat(head_1, ..., head_h)W^O**

onde W^O é uma matriz de pesos aprendida.

3- Codificador e Decodificador

No codificador, cada camada consiste em atenção multi-cabeça seguida por uma rede neural feed-forward. A entrada passa pela atenção multi-cabeça e é então somada à entrada original (conexão residual) e normalizada. O resultado passa pela rede feed-forward, é somado à entrada e normalizado novamente.

No decodificador, temos uma camada adicional de atenção multi-cabeça que leva a saída do codificador como K e V. Isso permite que o decodificador leve em consideração a entrada inteira ao produzir cada token de saída.

In [ ]:
modelo = TimeSeriesTransformerForPrediction(config)

### Pré-processamento dos dados

In [ ]:
# Função para criar a transformação (sequência de dados)
def cria_transformacao(freq: str, config: PretrainedConfig) -> Transformation:

    remove_field_names = []

    if config.num_static_real_features == 0:
        remove_field_names.append(FieldName.FEAT_STATIC_REAL)

    if config.num_dynamic_real_features == 0:
        remove_field_names.append(FieldName.FEAT_DYNAMIC_REAL)

    if config.num_static_categorical_features == 0:
        remove_field_names.append(FieldName.FEAT_STATIC_CAT)

    return Chain(

        # Passo 1: Remove campos estáticos/dinâmicos se não for especificado
        [RemoveFields(field_names = remove_field_names)]

        # Passo 2: Converte os dados para formato NumPy
        + (
            [
                AsNumpyArray(field = FieldName.FEAT_STATIC_CAT, expected_ndim = 1, dtype = int)
            ]
            if config.num_static_categorical_features > 0
            else []
        )
        + (
            [
                AsNumpyArray(field = FieldName.FEAT_STATIC_REAL, expected_ndim = 1)
            ]
            if config.num_static_real_features > 0
            else []
        )
        + [
            AsNumpyArray(field = FieldName.TARGET, expected_ndim = 1 if config.input_size == 1 else 2,
            ),

            # Passo 3: Trata os NaN's preenchendo o alvo com zero e retornando a máscara
            AddObservedValuesIndicator(target_field = FieldName.TARGET, output_field = FieldName.OBSERVED_VALUES),

            # Passo 4: Adiciona recursos temporais com base na freq do mês do ano do conjunto de dados
            # no caso em que freq="M" eles servem como codificações posicionais
            AddTimeFeatures(
                start_field = FieldName.START,
                target_field = FieldName.TARGET,
                output_field = FieldName.FEAT_TIME,
                time_features = time_features_from_frequency_str(freq),
                pred_length = config.prediction_length,
            ),

            # Passo 5: Adiciona outro recurso temporal (apenas um único número)
            # Informa ao modelo onde está o valor da série temporal, uma espécie de contador em execução
            AddAgeFeature(
                target_field = FieldName.TARGET,
                output_field = FieldName.FEAT_AGE,
                pred_length = config.prediction_length,
                log_scale = True,
            ),

            # Passo 6: Empilha verticalmente todos os recursos temporais na chave FEAT_TIME
            VstackFeatures(
                output_field = FieldName.FEAT_TIME,
                input_fields = [FieldName.FEAT_TIME, FieldName.FEAT_AGE]
                + (
                    [FieldName.FEAT_DYNAMIC_REAL]
                    if config.num_dynamic_real_features > 0
                    else []
                ),
            ),

            # Passo 7: Renomeia para corresponder aos nomes no dataset extraído do HuggingFace
            RenameFields(
                mapping = {
                    FieldName.FEAT_STATIC_CAT: "static_categorical_features",
                    FieldName.FEAT_STATIC_REAL: "static_real_features",
                    FieldName.FEAT_TIME: "time_features",
                    FieldName.TARGET: "values",
                    FieldName.OBSERVED_VALUES: "observed_mask",
                }
            ),
        ]
    )

## Divisão nas Amostras de Treino, Validação e Teste

In [ ]:
# Define uma função para criar um divisor de instâncias e separar os dados em conjuntos de treino, validação e teste
def create_instance_splitter(
    config: PretrainedConfig,
    mode: str,
    train_sampler: Optional[InstanceSampler] = None,
    validation_sampler: Optional[InstanceSampler] = None,
) -> Transformation:

    # Garante que o modo especificado seja um dos modos aceitos: treino, validação ou teste
    assert mode in ["train", "validation", "test"]

    # Define um dicionário mapeando cada modo para um sampler específico ou utiliza samplers padrões baseados na configuração
    instance_sampler = {
        "train": train_sampler
        or ExpectedNumInstanceSampler(

            # Sampler para treino, esperando um número específico de instâncias com um mínimo de pontos futuros
            num_instances = 1.0, min_future = config.prediction_length
        ),
        "validation": validation_sampler

        # Sampler para validação, dividindo com base em um mínimo de pontos futuros
        or ValidationSplitSampler(min_future = config.prediction_length),

        # Sampler para teste, sem necessidade de pontos futuros específicos
        "test": TestSplitSampler(),
    }[mode]

    # Cria e retorna um divisor de instâncias configurado com o sampler adequado e outras definições relevantes
    return InstanceSplitter(

        # Campo alvo para a previsão
        target_field = "values",

        # Campo que indica se um ponto de dados é um preenchimento (padding)
        is_pad_field = FieldName.IS_PAD,

        # Campo que indica o início da série temporal
        start_field = FieldName.START,

        # Campo que indica o início da previsão
        forecast_start_field = FieldName.FORECAST_START,

        # O sampler de instâncias escolhido baseado no modo
        instance_sampler = instance_sampler,

        # Define o comprimento do contexto passado
        past_length = config.context_length + max(config.lags_sequence),

        # Define o comprimento da previsão futura
        future_length = config.prediction_length,

        # Campos adicionais da série temporal a serem incluídos
        time_series_fields = ["time_features", "observed_mask"],
    )

## Criação dos Dataloaders

In [ ]:
# Define uma função para criar um DataLoader de treinamento com base nas configurações e dados fornecidos
def cria_dataloader_treino(config: PretrainedConfig,
                               freq,
                               data,
                               batch_size: int,
                               num_batches_per_epoch: int,
                               shuffle_buffer_length: Optional[int] = None,
                               cache_data: bool = True,
                               **kwargs) -> Iterable:

    # Inicia a lista de nomes de entradas para previsão baseando-se na configuração de características estáticas e temporais
    PREDICTION_INPUT_NAMES = ["past_time_features", "past_values", "past_observed_mask", "future_time_features"]

    # Adiciona o nome da característica categórica estática à lista se presente na configuração
    if config.num_static_categorical_features > 0:
        PREDICTION_INPUT_NAMES.append("static_categorical_features")

    # Adiciona o nome da característica real estática à lista se presente na configuração
    if config.num_static_real_features > 0:
        PREDICTION_INPUT_NAMES.append("static_real_features")

    # Estende a lista de nomes de entradas para incluir os dados de treinamento específicos
    TRAINING_INPUT_NAMES = PREDICTION_INPUT_NAMES + [
        "future_values",
        "future_observed_mask",
    ]

    # Cria a transformação dos dados com base na frequência e configuração fornecida
    transformation = cria_transformacao(freq, config)

    # Aplica a transformação nos dados no modo de treinamento
    transformed_data = transformation.apply(data, is_train = True)

    # Se solicitado, armazena em cache os dados transformados para otimizar o treinamento
    if cache_data:
        transformed_data = Cached(transformed_data)

    # Cria um divisor de instâncias para o treinamento que define como as janelas de dados serão amostradas
    instance_splitter = create_instance_splitter(config, "train")

    # Cria um fluxo cíclico dos dados transformados para amostragem contínua
    stream = Cyclic(transformed_data).stream()

    # Aplica o divisor de instâncias ao fluxo de dados para gerar instâncias de treinamento
    training_instances = instance_splitter.apply(stream, is_train = True)

    # Retorna os lotes empilhados das instâncias de treinamento conforme especificado
    return as_stacked_batches(
        training_instances,
        batch_size = batch_size,
        shuffle_buffer_length = shuffle_buffer_length,  # Define o comprimento do buffer de embaralhamento, se aplicável
        field_names = TRAINING_INPUT_NAMES,  # Especifica os nomes dos campos a serem incluídos nos lotes
        output_type = torch.tensor,  # Define o tipo de saída dos lotes
        num_batches_per_epoch = num_batches_per_epoch,  # Especifica o número de lotes por época
    )

    # Define uma função para criar um DataLoader de teste a partir de configurações pré-definidas e dados
def cria_dataloader_teste(config: PretrainedConfig, freq, data, batch_size: int, **kwargs):

    # Define os nomes das entradas necessárias para previsão baseadas na configuração
    PREDICTION_INPUT_NAMES = ["past_time_features", "past_values", "past_observed_mask", "future_time_features"]

    # Se houver características categóricas estáticas, adiciona ao conjunto de entradas de previsão
    if config.num_static_categorical_features > 0:
        PREDICTION_INPUT_NAMES.append("static_categorical_features")

    # Se houver características reais estáticas, adiciona ao conjunto de entradas de previsão
    if config.num_static_real_features > 0:
        PREDICTION_INPUT_NAMES.append("static_real_features")

    # Cria a transformação a ser aplicada nos dados com base na frequência e configuração
    transformation = cria_transformacao(freq, config)

    # Aplica a transformação aos dados no modo de teste (não treinamento)
    transformed_data = transformation.apply(data, is_train=False)

    # Cria um amostrador de instâncias para o modo de teste que irá selecionar a última janela de contexto vista durante o treinamento
    instance_sampler = create_instance_splitter(config, "test")

    # Aplica o amostrador de instâncias aos dados transformados no modo de teste
    testing_instances = instance_sampler.apply(transformed_data, is_train = False)

    # Retorna os lotes empilhados como tensores PyTorch, com um tamanho de lote especificado e usando os nomes de campos definidos
    return as_stacked_batches(
        testing_instances,
        batch_size = batch_size,
        output_type = torch.tensor,
        field_names = PREDICTION_INPUT_NAMES,
    )

In [ ]:
# Cria o dataloader de treino
dl_treino = cria_dataloader_treino(config = config,
                                  freq = freq,
                                  data = dataset_treino,
                                  batch_size = 256,
                                   num_batches_per_epoch = 100)

In [ ]:
# Imprime as chaves de um batch de dados
batch = next(iter(dl_treino))
for k, v in batch.items():
    print(k)

In [ ]:
# Imprime as chaves e respectivos valores de um batch de dados
batch = next(iter(dl_treino))
for k, v in batch.items():
    print(k, v)

In [ ]:
# Cria o dataloader de teste
dl_teste = cria_dataloader_teste(config = config,
                                freq = freq,
                                data = dataset_teste,
                                batch_size = 64)

## Loop de Treinamento do Modelo

In [ ]:
# Cria o acelerador
accelerator = Accelerator()

# Registra o device
device = accelerator.device

device

In [ ]:
# Envia o modelo para o device
modelo.to(device)

In [ ]:
# Otimizador
optimizer = AdamW(modelo.parameters(), lr = 6e-4, betas = (0.9, 0.95), weight_decay = 1e-1)

In [ ]:
# Carrega o modelo, o otimizador e o dataloader de treino
modelo, optimizer, dl_treino = accelerator.prepare(modelo, optimizer, dl_treino)

### Matemática da Otimização do Modelo

Aqui estão as etapas do AdamW em termos matemáticos:

**Cálculo dos Momentos de Primeira e Segunda Ordem**

Para cada parâmetro θ, Adam mantém uma estimativa do primeiro momento (a média móvel dos gradientes passados) e do segundo momento (a média móvel dos quadrados dos gradientes passados). Para uma dada etapa t, gradiente g_t e parâmetros de decaimento β1 e β2, essas estimativas são atualizadas da seguinte forma:

- m_t = β1 * m_(t-1) + (1 - β1) * g_t
- v_t = β2 * v_(t-1) + (1 - β2) * g_t^2

**Correção de Viés**

Como m_t e v_t são inicializados como zero, eles são tendenciosos para zero no início do treinamento. Portanto, Adam realiza uma correção de viés para compensar isso:

- m_t_hat = m_t / (1 - β1^t)
- v_t_hat = v_t / (1 - β2^t)

**Atualização de Peso**

Finalmente, os pesos são atualizados com uma taxa de aprendizado η e um termo de decaimento de peso w:

**θ = θ - η * (m_t_hat / (sqrt(v_t_hat) + ε) + w * θ)**

Onde ε é um termo de suavização para evitar a divisão por zero (geralmente algo como 1e-8).

AdamW difere do Adam na forma como o termo de decaimento de peso é aplicado. No Adam original, o decaimento de peso é aplicado antes do cálculo do gradiente, o que pode levar a um acoplamento entre a atualização do peso e a escala do gradiente. AdamW aplica o decaimento de peso diretamente na etapa de atualização do peso, o que "desacopla" a regularização do decaimento de peso da escala do gradiente.

O cálculo das derivadas é uma parte fundamental do treinamento de redes neurais e é aplicado durante o processo de retropropagação (backpropagation), que é usado para atualizar os pesos da rede. No contexto do otimizador AdamW (ou qualquer otimizador baseado em gradiente), a derivada é usada para calcular o gradiente da função de perda com relação a cada peso na rede.

Na prática, você não precisa calcular essas derivadas manualmente. Frameworks modernos de aprendizado profundo, como TensorFlow e PyTorch, usam diferenciação automática para calcular as derivadas. Você simplesmente define a função de perda e o framework cuida de calcular os gradientes para você.

No caso específico do AdamW, a derivada é usada para calcular o gradiente g_t na etapa de atualização de momentos. Esse gradiente é simplesmente a derivada da função de perda com relação ao peso específico que está sendo atualizado. O gradiente indica a direção e a magnitude da mudança no peso que resultará no maior decréscimo na função de perda.

Então, em resumo, a derivada é usada no cálculo do gradiente, que é então usado para atualizar os pesos da rede neural na direção que minimiza a função de perda. Queremos os pesos que levem ao menor erro possível.

---

O otimizador é uma parte crítica do treinamento de redes neurais, pois é responsável por atualizar os pesos das conexões na rede para minimizar o erro entre as previsões do modelo e os dados reais.

Neste caso, o otimizador escolhido foi o AdamW, que é uma variação do otimizador Adam com regularização de decaimento de peso (weight decay). Vamos analisar cada componente:


model.parameters(): Esta é uma função que retorna todos os parâmetros (pesos e vieses) do modelo que estão sendo otimizados.


lr = 6e-4: Este é o valor da taxa de aprendizado. A taxa de aprendizado controla a rapidez com que o modelo aprende. Uma taxa de aprendizado muito alta pode fazer com que o modelo salte sobre o mínimo global, enquanto uma taxa de aprendizado muito baixa pode fazer com que o modelo aprenda muito lentamente.

betas = (0.9, 0.95): Estes são os coeficientes usados para calcular as médias móveis dos gradientes e dos quadrados dos gradientes, respectivamente. Os valores padrão para Adam são geralmente (0.9, 0.999), mas eles podem ser ajustados.

weight_decay = 1e-1: Este é o termo de regularização de decaimento de peso. O decaimento de peso é uma técnica de regularização que impõe uma penalidade sobre a magnitude dos pesos na rede. O objetivo é prevenir o overfitting, penalizando pesos grandes e incentivando pesos menores, levando a um modelo mais simples.

O otimizador AdamW combina as vantagens do método de otimização adaptativa de Adam, que se adapta ao longo do tempo para otimizar a taxa de aprendizado de cada parâmetro individual, com a regularização de decaimento de peso para melhorar o desempenho do modelo em conjuntos de dados de teste.



AdamW é um algoritmo de otimização baseado em gradientes usado para atualizar os pesos da rede neural para minimizar a função de perda. AdamW é uma extensão do Adam com decaimento de peso, como descrito em "Decoupled Weight Decay Regularization" por Loshchilov e Hutter.

In [ ]:
%%time

modelo.train()

# Inicia o loop de treinamento para um número pré-definido de épocas
for epoch in range(10):

    # Itera sobre os lotes do DataLoader de treinamento
    for idx, batch in enumerate(dl_treino):

        # Zera os gradientes do otimizador para evitar acumulação de gradientes de iterações anteriores
        optimizer.zero_grad()

        # Gera as saídas do modelo passando as características adequadas do lote atual
        outputs = modelo(

            # Passa características categóricas estáticas para o dispositivo se configurado
            static_categorical_features = batch["static_categorical_features"].to(device)
            if config.num_static_categorical_features > 0
            else None,

            # Passa características reais estáticas para o dispositivo se configurado
            static_real_features = batch["static_real_features"].to(device)
            if config.num_static_real_features > 0
            else None,

            # Passa características temporais passadas para o dispositivo
            past_time_features = batch["past_time_features"].to(device),

            # Passa valores passados para o dispositivo
            past_values = batch["past_values"].to(device),

            # Passa características temporais futuras para o dispositivo
            future_time_features = batch["future_time_features"].to(device),

            # Passa valores futuros para o dispositivo (usado para treinamento supervisionado)
            future_values = batch["future_values"].to(device),

            # Passa a máscara de observações passadas para o dispositivo
            past_observed_mask = batch["past_observed_mask"].to(device),

            # Passa a máscara de observações futuras para o dispositivo
            future_observed_mask = batch["future_observed_mask"].to(device),
        )

        # Atribui a perda calculada pelas saídas do modelo
        loss = outputs.loss

        # Realiza a retropropagação do erro para ajustar os pesos do modelo
        accelerator.backward(loss)

        # Atualiza os pesos do modelo com base nos gradientes calculados
        optimizer.step()

        # A cada 100 lotes, imprime o erro atual do modelo
        if idx % 100 == 0:
            print("Erro do Modelo:", loss.item())

## Avaliação do modelo

In [ ]:
modelo.eval()

In [ ]:
forecasts = []

In [ ]:
# Itera sobre os lotes do DataLoader de teste
for batch in dl_teste:

    # Gera previsões usando o modelo para o lote atual, passando as características conforme a configuração
    outputs = modelo.generate(

        # Passa características categóricas estáticas para o dispositivo se houver alguma
        static_categorical_features = batch["static_categorical_features"].to(device)
        if config.num_static_categorical_features > 0
        else None,

        # Passa características reais estáticas para o dispositivo se houver alguma
        static_real_features = batch["static_real_features"].to(device)
        if config.num_static_real_features > 0
        else None,

        # Passa características temporais passadas para o dispositivo
        past_time_features = batch["past_time_features"].to(device),

        # Passa valores passados para o dispositivo
        past_values = batch["past_values"].to(device),

        # Passa características temporais futuras para o dispositivo
        future_time_features = batch["future_time_features"].to(device),

        # Passa a máscara de observações passadas para o dispositivo
        past_observed_mask = batch["past_observed_mask"].to(device),
    )

    # Adiciona as sequências de previsões geradas à lista de previsões, movendo para a CPU e convertendo para numpy
    forecasts.append(outputs.sequences.cpu().numpy())

In [ ]:
# Shape das previsões
forecasts[0].shape

In [ ]:
# Ajuste do shape
forecasts = np.vstack(forecasts)
print(forecasts.shape)

### Calculando as Métricas MASE e SMAPE

As métricas MASE (Erro Médio Absoluto Escalonado) e SMAPE (Erro Percentual Absoluto Médio Simétrico) são usadas para avaliar a precisão das previsões em problemas de séries temporais. Ambas são métricas que tentam colocar o erro de previsão em um contexto mais fácil de interpretar, escalonando ou normalizando o erro de alguma forma.

MASE (Erro Médio Absoluto Escalonado): A MASE é uma métrica que compara o erro de previsão de um método de previsão com o erro de previsão de um método de previsão "ingênuo" (naive), geralmente uma previsão que simplesmente assume que o valor futuro será o mesmo que o valor atual. A MASE é calculada como a média dos erros absolutos das previsões dividida pela média dos erros absolutos das previsões ingênuas. Um valor MASE de 1 indica que o modelo tem o mesmo erro médio que a previsão ingênua, enquanto um valor MASE menor que 1 indica que o modelo é melhor que a previsão ingênua.

SMAPE (Erro Percentual Absoluto Médio Simétrico): A SMAPE é uma métrica que expressa o erro de previsão como uma porcentagem do valor verdadeiro e da previsão. Ao contrário do erro percentual absoluto médio (MAPE), a SMAPE tem um denominador que incorpora tanto o valor verdadeiro quanto o valor previsto, tornando-a "simétrica". Isso evita alguns dos problemas com o MAPE, onde previsões muito baixas podem levar a erros percentuais muito grandes. A SMAPE varia de 0 a 200, onde 0 indica que não há erro (previsão perfeita) e 200 indica que a previsão está completamente errada.

Ambas as métricas têm suas próprias vantagens e limitações, e a escolha entre elas dependerá do problema específico e das características dos dados.


In [ ]:
# Funções com os cálculos matemáticos para as métricas

# MASE
def mase_score(pred, real, train, m):
    num = np.mean(np.abs(pred - real))
    denom = np.mean(np.abs(train[m:] - train[:-m]))
    return num / denom if denom != 0 else np.nan

# MAPE
def smape_score(pred, real):
    return 100 * np.mean(2 * np.abs(pred - real) / (np.abs(pred) + np.abs(real) + 1e-8))

In [ ]:
# Mediana das previsões
forecast_median = np.median(forecasts, 1)

In [ ]:
# Listas das métricas
mase_metrics = []
smape_metrics = []

# Inicializa o loop para percorrer os itens do conjunto de teste
for item_id, ts in enumerate(dataset_teste):

    # Separa os dados de treinamento excluindo o comprimento da previsão do final
    training_data = ts["target"][:-prediction_length]

    # Separa os dados reais (ground truth) para o comprimento da previsão
    ground_truth = ts["target"][-prediction_length:]

    # Calcula MASE manualmente
    mase = mase_score(forecast_median[item_id], np.array(ground_truth), np.array(training_data), get_seasonality(freq))
    mase_metrics.append(mase)

    # Calcula sMAPE manualmente
    smape = smape_score(forecast_median[item_id], np.array(ground_truth))
    smape_metrics.append(smape)

In [ ]:
print(f"MASE: {np.mean(mase_metrics)}")
print(f"sMAPE: {np.mean(smape_metrics)}")


In [ ]:
# Plot
plt.scatter(mase_metrics, smape_metrics, alpha = 0.3)
plt.xlabel("MASE")
plt.ylabel("sMAPE")
plt.show()

In [ ]:
# Define a função para plotar a série temporal e previsões para um índice específico
def plot(ts_index):

    # Cria uma figura e um eixo para o plot
    fig, ax = plt.subplots(figsize = (10, 5))

    # Gera o índice de datas para a série temporal a partir dos metadados e do comprimento do alvo
    index = pd.period_range(start = dataset_teste[ts_index][FieldName.START],
                            periods = len(dataset_teste[ts_index][FieldName.TARGET]),
                            freq = freq).to_timestamp()

    # Configura os locais principais do eixo x para serem nos meses de janeiro e julho
    ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 7)))

    # Configura os locais secundários do eixo x para serem em todos os meses
    ax.xaxis.set_minor_locator(mdates.MonthLocator())

    # Plota os valores reais da série temporal para os últimos 2 períodos de previsão
    ax.plot(index[-2*prediction_length:],
            dataset_teste[ts_index]["target"][-2*prediction_length:],
            label="Valor Real")

    # Plota a mediana das previsões para o último período de previsão
    plt.plot(index[-prediction_length:],
             np.median(forecasts[ts_index], axis=0),
             label = "Mediana das Previsões")

    # Preenche a área entre a média menos o desvio padrão e a média mais o desvio padrão das previsões
    plt.fill_between(
        index[-prediction_length:],
        forecasts[ts_index].mean(0) - forecasts[ts_index].std(axis=0),
        forecasts[ts_index].mean(0) + forecasts[ts_index].std(axis=0),
        alpha = 0.3,
        interpolate = True,
        label = "+/- 1-std",
    )

    # Adiciona uma legenda ao plot
    plt.legend()

    # Exibe o plot
    plt.show()

In [ ]:
# Previsão para o índice 334
plot(334)